In [18]:
!pip install ekphrasis

In [19]:
%env WANDB_PROJECT=nlu_sentiment_analysis
import wandb
wandb.login()

env: WANDB_PROJECT=nlu_sentiment_analysis


True

In [20]:
from ekphrasis.classes.preprocessor import TextPreProcessor
from ekphrasis.classes.tokenizer import SocialTokenizer
from ekphrasis.dicts.emoticons import emoticons

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import itertools
import requests, re, string, datetime

PATH = "/content/drive/MyDrive/Colab Notebooks/NLU/NLU Project/"

In [21]:
train = pd.read_csv(PATH + "data/Sentipolc16/training_set_sentipolc16.csv")
train.head()

,idtwitter,subj,opos,oneg,iro,lpos,lneg,top,text
0,122449983151669248,1,0,1,0,0,1,1,Intanto la partita per Via Nazionale si compli...
1,125485104863780865,1,0,1,0,0,1,1,"False illusioni, sgradevoli realtà Mario Monti..."
2,125513454315507712,1,0,1,0,0,1,1,"False illusioni, sgradevoli realtà #editoriale..."
3,125524238290522113,1,0,1,0,0,1,1,Mario Monti: Berlusconi risparmi all'Italia il...
4,125527933224886272,1,0,1,0,0,1,1,Mario Monti: Berlusconi risparmi all'Italia il...


In [22]:
file1 = open(PATH + "data/Sentipolc16/test_set_sentipolc16_gold2000.csv", 'r') 
Lines = file1.readlines()
 
test = []
for line in Lines:
  arr = line.split("\",")
  if len(arr) != 9:
    arr[8] = arr[8] + arr[9]  #to account for tweets containing the delimiter charachter that would create more splits than needed
    del arr[9:]
  for i in range(8):
    arr[i] = int(arr[i].strip("\""))
  test.append(arr)

test = pd.DataFrame(test, columns=train.columns)
test.head()

,idtwitter,subj,opos,oneg,iro,lpos,lneg,top,text
0,507074506880712705,0,0,0,0,0,0,2,"""Tra 5 minuti presentazione piano scuola del g..."
1,507075789456961536,1,1,0,0,1,0,2,"""\""@matteorenzi: Alle 10 appuntamento su http:..."
2,507077511902425088,1,0,1,0,0,1,2,"""#labuonascuola gli #evangelisti #digitali non..."
3,507079183315787777,0,0,0,0,0,0,2,"""Riforma scuola Tutto il discorso di Renzi su..."
4,507080190225563648,1,0,0,0,0,0,2,""".@matteorenzi @MiurSocial #labuonascuola bast..."


In [23]:
text_processor = TextPreProcessor (
    # terms that will be normalized
    normalize=[ 'url' , 'email', 'user', 'percent', 'money', 'phone', 'time', 'date', 'number'] ,
    # terms that will be annotated
    annotate={"hashtag"} ,
    fix_html=True ,  # fix HTML tokens

    unpack_hashtags=True ,  # perform word segmentation on hashtags

    # select a tokenizer. You can use SocialTokenizer, or pass your own
    # the tokenizer, should take as input a string and return a list of tokens
    tokenizer=SocialTokenizer(lowercase=True).tokenize,
    dicts = [ emoticons ]
)

sentences = train["text"]
labels = train.iloc[:,1]

examples = []
for i , s in enumerate(sentences):
    s = s.lower()
    s = str(" ".join(text_processor.pre_process_doc(s)))
    s = re.sub(r"[^a-zA-ZÀ-ú</>!?♥♡\s\U00010000-\U0010ffff]", ' ', s)
    s = re.sub(r"\s+", ' ', s)
    s = re.sub(r'(\w)\1{2,}',r'\1\1', s)
    s = re.sub ( r'^\s' , '' , s )
    s = re.sub ( r'\s$' , '' , s )
    examples.append([labels[i],s])
    i = i+1
examples = np.array(examples)


sentences_test = test["text"]
test_ids = list(test.index)

examples_test = []
for i , s in enumerate(sentences_test):
    s = s.lower()
    s = str(" ".join(text_processor.pre_process_doc(s)))
    s = re.sub(r"[^a-zA-ZÀ-ú</>!?♥♡\s\U00010000-\U0010ffff]", ' ', s)
    s = re.sub(r"\s+", ' ', s)
    s = re.sub(r'(\w)\1{2,}',r'\1\1', s)
    s = re.sub ( r'^\s' , '' , s )
    s = re.sub ( r'\s$' , '' , s )
    examples_test.append([labels[i],s])
    i = i+1
examples_test = np.array(examples_test)

Reading english - 1grams ...
Reading english - 2grams ...
Reading english - 1grams ...


#### tokenizer.py

In [24]:
!pip install tqdm boto3 requests regex sentencepiece sacremoses pytorch-transformers
!pip install transformers
!pip install datasets

In [25]:
"""Tokenization classes for Italian AlBERTo models."""
import collections
import logging
import os
import re

from ekphrasis.classes.preprocessor import TextPreProcessor
from ekphrasis.classes.tokenizer import SocialTokenizer
from ekphrasis.dicts.emoticons import emoticons

import numpy as np

from transformers import BertTokenizer, WordpieceTokenizer

def separate2united_labels(row):
  """
  Return a single scalar integer label associated to the polarity of the tweet.

  Negative -> 0
  Neutral  -> 1
  Positive -> 2
  Mixed    -> 3
  """
  if row["opos"] == 0 and row["oneg"] == 0:
    return 1
  elif row["oneg"] == 0 and row["opos"] == 1:
    return 2
  elif row["oneg"] == 1 and row["opos"] == 0:
    return 0
  else:
    return 3

def load_vocab(vocab_file):
    """Loads a vocabulary file into a dictionary."""
    vocab = collections.OrderedDict()
    with open(vocab_file, "r", encoding="utf-8") as reader:
        tokens = reader.readlines()
    for index, token in enumerate(tokens):
        token = token.rstrip("\n")
        vocab[token] = index
    return vocab

text_processor = TextPreProcessor(
    # terms that will be normalized
    normalize=['url', 'email', 'user', 'percent', 'money', 'phone', 'time', 'date', 'number'],
    # terms that will be annotated
    annotate={"hashtag"},
    fix_html=True,  # fix HTML tokens

    unpack_hashtags=True,  # perform word segmentation on hashtags

    # select a tokenizer. You can use SocialTokenizer, or pass your own
    # the tokenizer, should take as input a string and return a list of tokens
    tokenizer=SocialTokenizer(lowercase=True).tokenize,
    dicts=[emoticons]
)

class AlBERTo_Preprocessing(object):
    def __init__(self, do_lower_case=True, **kwargs):
        self.do_lower_case = do_lower_case

    def preprocess(self, text):
        if self.do_lower_case:
            text = text.lower()
        text = str(" ".join(text_processor.pre_process_doc(text)))
        text = re.sub(r'[^a-zA-ZÀ-ú</>!?♥♡\s\U00010000-\U0010ffff]', ' ', text)
        text = re.sub(r'\s+', ' ', text)
        text = re.sub(r'(\w)\1{2,}', r'\1\1', text)
        text = re.sub(r'^\s', '', text)
        text = re.sub(r'\s$', '', text)
        return text

class AlBERToTokenizer(BertTokenizer):

    def __init__(self, vocab_file, do_lower_case=True,
                 do_basic_tokenize=True, do_char_tokenize=False, do_wordpiece_tokenize=False, do_preprocessing = True, unk_token='[UNK]',
                 sep_token='[SEP]',
                 pad_token='[PAD]', cls_token='[CLS]', mask_token='[MASK]', **kwargs):
        super(BertTokenizer, self).__init__(
            unk_token=unk_token, sep_token=sep_token, pad_token=pad_token,
            cls_token=cls_token, mask_token=mask_token, **kwargs)

        self.do_wordpiece_tokenize = do_wordpiece_tokenize
        self.lower_case = do_lower_case
        self.vocab_file = vocab_file
        self.do_basic_tokenize = do_basic_tokenize
        self.do_char_tokenize = do_char_tokenize
        self.unk_token = unk_token
        self.do_preprocessing = do_preprocessing

        if not os.path.isfile(vocab_file):
            raise ValueError(
                "Can't find a vocabulary file at path '{}'.".format(vocab_file))

        self.vocab = load_vocab(vocab_file)
        self.ids_to_tokens = collections.OrderedDict(
            [(ids, tok) for tok, ids in self.vocab.items()])

        if do_wordpiece_tokenize:
            self.wordpiece_tokenizer = WordpieceTokenizer(vocab=self.vocab,
                                                          unk_token=self.unk_token)
            
        self.base_bert_tok = BertTokenizer(vocab_file=self.vocab_file, do_lower_case=do_lower_case,
                                      unk_token=unk_token, sep_token=sep_token, pad_token=pad_token,
                                      cls_token=cls_token, mask_token=mask_token, **kwargs)

    def _convert_token_to_id(self, token):
        """Converts a token (str/unicode) to an id using the vocab."""
        # if token[:2] == '##':
        #     token = token[2:]

        return self.vocab.get(token, self.vocab.get(self.unk_token))

    def convert_token_to_id(self, token):
        return self._convert_token_to_id(token)

        return self.vocab.get(token, self.vocab.get(self.unk_token))

    def _convert_id_to_token(self, id):
        # if token[:2] == '##':
        #     token = token[2:]

        return list(self.vocab.keys())[int(id)]
    def convert_id_to_token(self, id):
        return self._convert_id_to_token(id)

    def _convert_tokens_to_string(self,tokens):
        """Converts a sequence of tokens (string) to a single string."""
        out_string = ' '.join(tokens).replace('##', '').strip()
        return out_string

    def convert_tokens_to_string(self,tokens):
        return self._convert_tokens_to_string(tokens)

    def _tokenize(self, text, never_split=None, **kwargs):
        if self.do_preprocessing:
            if self.lower_case:
                text = text.lower()
            text = str(" ".join(text_processor.pre_process_doc(text)))
            text = re.sub(r'[^a-zA-ZÀ-ú</>!?♥♡\s\U00010000-\U0010ffff]', ' ', text)
            text = re.sub(r'\s+', ' ', text)
            text = re.sub(r'(\w)\1{2,}', r'\1\1', text)
            text = re.sub(r'^\s', '', text)
            text = re.sub(r'\s$', '', text)
            # print(s)

        split_tokens = [text]
        if self.do_wordpiece_tokenize:
            wordpiece_tokenizer = WordpieceTokenizer(self.vocab,self.unk_token)
            split_tokens = wordpiece_tokenizer.tokenize(text)

        elif self.do_char_tokenize:
            tokenizer = CharacterTokenizer(self.vocab, self.unk_token)
            split_tokens = tokenizer.tokenize(text)

        elif self.do_basic_tokenize:
            """Tokenizes a piece of text."""
            split_tokens = self.base_bert_tok.tokenize(text)

        return split_tokens

    def tokenize(self, text, never_split=None, **kwargs):
        return self._tokenize(text, never_split)


class CharacterTokenizer(object):
    """Runs Character tokenziation."""

    def __init__(self, vocab, unk_token,
                 max_input_chars_per_word=100, with_markers=True):
        """Constructs a CharacterTokenizer.
        Args:
            vocab: Vocabulary object.
            unk_token: A special symbol for out-of-vocabulary token.
            with_markers: If True, "#" is appended to each output character except the
                first one.
        """
        self.vocab = vocab
        self.unk_token = unk_token
        self.max_input_chars_per_word = max_input_chars_per_word
        self.with_markers = with_markers

    def tokenize(self, text):
        """Tokenizes a piece of text into characters.

        For example:
            input = "apple"
            output = ["a", "##p", "##p", "##l", "##e"]  (if self.with_markers is True)
            output = ["a", "p", "p", "l", "e"]          (if self.with_markers is False)
        Args:
            text: A single token or whitespace separated tokens.
                This should have already been passed through `BasicTokenizer`.
        Returns:
            A list of characters.
        """

        output_tokens = []
        for i, char in enumerate(text):
            if char not in self.vocab:
                output_tokens.append(self.unk_token)
                continue

            if self.with_markers and i != 0:
                output_tokens.append('##' + char)
            else:
                output_tokens.append(char)

        return output_tokens

a = AlBERTo_Preprocessing(do_lower_case=True)
s = "#IlGOverno presenta le linee guida sulla scuola #labuonascuola - http://t.co/SYS1T9QmQN"
b = a.preprocess(s)
print(b)

#c = AlBERToTokenizer(do_lower_case=True,vocab_file=VOCAB_FILE, do_preprocessing=True)
#d = c.tokenize(s)
#print(d)

Reading english - 1grams ...
Reading english - 2grams ...
Reading english - 1grams ...
<hashtag> il governo </hashtag> presenta le linee guida sulla scuola <hashtag> la buona scuola </hashtag> <url>


#### Pytorch

In [26]:
from transformers import AutoTokenizer, AutoModel

a = AlBERTo_Preprocessing(do_lower_case=True)
s: str = "#IlGOverno presenta le linee guida sulla scuola #labuonascuola - http://t.co/SYS1T9QmQN"
b = a.preprocess(s)

tok = AutoTokenizer.from_pretrained("m-polignano-uniba/bert_uncased_L-12_H-768_A-12_italian_alb3rt0")
model = AutoModel.from_pretrained("m-polignano-uniba/bert_uncased_L-12_H-768_A-12_italian_alb3rt0")

tok.model_max_length = 128 #model.config.max_position_embeddings
tokens = tok.tokenize(b)
print(tokens)

Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file https://huggingface.co/m-polignano-uniba/bert_uncased_L-12_H-768_A-12_italian_alb3rt0/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/ddbe3a15a091d937ecbe95bd9c6fc3c82a8c0fde2b2725504806bae3d5b553c7.ae4dca69541bd7de0c171dd17916268b75b9de1cdbd9670add7a62ad5b98a01e
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "directionality": "bidi",
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head":

['<', 'ha', '##shtag', '>', 'il', 'governo', '<', '/', 'ha', '##shtag', '>', 'presenta', 'le', 'linee', 'guida', 'sulla', 'scuola', '<', 'ha', '##shtag', '>', 'la', 'buona', 'scuola', '<', '/', 'ha', '##shtag', '>', '<', 'ur', '##l', '>']


In [27]:
output = model(**tok("ciao " * 1000, return_tensors="pt", truncation=True))

output[0][:,0,:].shape , output[1].shape , output[0][:,0,:10] , output[1][0][:10]

(torch.Size([1, 768]),
 torch.Size([1, 768]),
 tensor([[-0.6951, -0.8461, -0.6175, -0.8552,  0.0815,  1.5553,  0.7703, -0.3757,
           0.9055,  1.7895]], grad_fn=<SliceBackward>),
 tensor([-0.6683,  0.9256, -0.5889, -0.0618,  0.6870,  0.0903, -0.0759,  0.7948,
         -0.5468,  0.2115], grad_fn=<SliceBackward>))

In [28]:
## Dataset
from datasets import Dataset
from transformers import TrainingArguments
from transformers import Trainer
from transformers.modeling_outputs import SequenceClassifierOutput

def tokenize_function(examples):
    sa = tok(examples["text"], padding="max_length", truncation=True)
    return sa

df = pd.DataFrame({"text": train.text.apply(a.preprocess), "idx": train.index, "labels": train[["opos", "oneg"]].apply(separate2united_labels, axis=1)})
dataset = Dataset.from_pandas(df)
print(dataset[0])

training = dataset.map(tokenize_function, batched=True).shuffle(seed=42).with_format("torch")

df = pd.DataFrame({"text": test.text.apply(a.preprocess), "idx": test.index, "labels": test[["opos", "oneg"]].apply(separate2united_labels, axis=1)})
dataset = Dataset.from_pandas(df)

testing = dataset.map(tokenize_function, batched=True).with_format("torch")


# for s in training[0].keys():
#   if type(training[0][s]) != list:
#     print(s, training[0][s])
#   else:
#     print(s, "len=" ,len(training[0][s]))

{'text': 'intanto la partita per via nazionale si complica <hashtag> sacco manni </hashtag> dice che mica tutti sono mario <hashtag> monti </hashtag> <url> via <user>', 'idx': 0, 'labels': 0}


In [29]:
# print(torch.cuda.get_device_name(0))
# print('Memory Usage:')
# print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
# print('Cached:   ', round(torch.cuda.memory_cached(0)/1024**3,1), 'GB')

In [30]:
training["token_type_ids"].element_size() * training["token_type_ids"].nelement() / 1000000 #, training["token_type_ids"].type(torch.int8) 

7.58784

In [31]:
## Model
import torch
import torchvision.transforms as T, torch.nn.functional as F, torch.nn as nn

class MyNet(nn.Module):
  """
    Attach a FC layer on top of the BERT head in order to produce a classification output.
    Hyperparameters are taken from Alberto.

    The pooled_output output of BERT is basically a projection of the [CLS] embeddings via another FC layer (768 -> 768 hidden units).
    We stack another FC layer with Dropout on top of that, as reported in https://github.com/google-research/bert/blob/eedf5716ce1268e56f0a50264a88cafad334ac61/run_classifier.py#L574
  """

  def __init__(self):
    super(MyNet, self).__init__()

    self.model = AutoModel.from_pretrained("m-polignano-uniba/bert_uncased_L-12_H-768_A-12_italian_alb3rt0")
    self.dropout = nn.Dropout(0.1)
    self.linear = nn.Linear(768, 4)

    self.loss_fct = nn.CrossEntropyLoss()


  def forward(self, labels, input_ids, **args):
    #For the output format -> https://huggingface.co/transformers/_modules/transformers/models/bert/modeling_bert.html#BertForSequenceClassification.forward
    outputs = self.model(input_ids, **args)
    x = self.dropout(outputs[1])
    logits = self.linear(x)
    
    loss = self.loss_fct(logits, labels)
  
    return SequenceClassifierOutput(
        loss=loss,
        logits=logits,
        hidden_states=outputs.hidden_states,
        attentions=outputs.attentions,
    )

#### Trainer loop

In [1]:
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='macro')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }


TRAIN_BATCH_SIZE = 32 #512
PREDICT_BATCH_SIZE = 32 #512
EVAL_BATCH_SIZE = 32 #512
LEARNING_RATE = 2e-5
NUM_TRAIN_EPOCHS = 3 #10
MAX_SEQ_LENGTH = 128

# Warmup is a period of time where hte learning rate 
# is small and gradually increases--usually helps training.
WARMUP_PROPORTION = 0.1
num_train_steps = int(len(training) / TRAIN_BATCH_SIZE * NUM_TRAIN_EPOCHS)+1
NUM_WARMUP_STEPS = int(NUM_TRAIN_EPOCHS * WARMUP_PROPORTION)

training_args = TrainingArguments("test_trainer", 
                                  num_train_epochs=3,
                                  per_device_train_batch_size=32,
                                  per_device_eval_batch_size=32,
                                  evaluation_strategy="epoch",
                                  learning_rate=LEARNING_RATE,
                                  warmup_steps=NUM_WARMUP_STEPS,
                                  weight_decay=0.01,
                                  adam_beta1=0.9,
                                  adam_beta2=0.999,
                                  adam_epsilon=1e-6,
                                  report_to="wandb",  
                                  run_name="alberto-repr-colab-2"
)

np.random.seed(0)
torch.manual_seed(0)

trainer = Trainer(
    model_init=MyNet, 
    args=training_args, 
    train_dataset=training , 
    eval_dataset=testing,
    compute_metrics=compute_metrics
)


trainer.train()
wandb.finish()

NameError: ignored

In [ ]:
1	No log	0.825890	0.681000	0.488477	0.503568	0.499365
2	No log	0.897935	0.657500	0.491635	0.553197	0.489481
3	0.843400	0.975949	0.645000	0.494844	0.524969	0.503557

#### explicit training loop

In [ ]:
from torch.utils.data import DataLoader

tokenized_datasets = training.remove_columns(["text", "idx"])
tokenized_datasets.set_format("torch")
train_dataloader = DataLoader(tokenized_datasets, shuffle=True, batch_size=8)

from transformers import AdamW

optimizer = AdamW(model.parameters(), lr=5e-5)

from transformers import get_scheduler

num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps
)

import torch

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model=MyNet()
model.to(device)

from tqdm.auto import tqdm

progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

Some weights of the model checkpoint at m-polignano-uniba/bert_uncased_L-12_H-768_A-12_italian_alb3rt0 were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


KeyboardInterrupt: ignored

In [ ]:
torch.cuda.empty_cache()